In [1]:
#import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json 
import csv
from datetime import datetime
from citipy import citipy as ctpy
from config import weather_api_key


In [2]:
#create set of random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)


In [3]:
#add the latitudes and longitudes to a list
coordinates=list(lat_lngs)

In [4]:
#create a list for holding the nearest cities
cities = []

#identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = ctpy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    #if the city is unique, add to the city list
    if city not in cities:
        cities.append(city)

In [5]:
#create empty list to hold weather data
city_data = []

#build basic URL for OpenWeatherMap
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

#loop through all cities in the list
for i, city in enumerate(cities):
    
    #create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    #run API request for each of the cities
    try:
        #parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        #parse out the needed data
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        city_weather_desc = city_weather['weather'][0]['description']
        city_data.append({"City": city.title(),
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_humidity,
                         "Cloudiness": city_clouds,
                         "Wind Speed": city_wind,
                         "Country": city_country,
                         "Current Description": city_weather_desc})
    except:
        pass
    


In [8]:
#change to dataframe
city_data_df = pd.DataFrame(city_data)


#change column order to desired format
new_column_order = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Current Description']
city_data_df = city_data_df[new_column_order]

city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Sitka,US,57.0531,-135.3300,67.89,64,20,4.61,few clouds
1,Bilma,NE,18.6853,12.9164,93.42,29,30,6.96,scattered clouds
2,Ushuaia,AR,-54.8000,-68.3000,42.46,49,40,13.80,scattered clouds
3,Talnakh,RU,69.4865,88.3972,59.38,82,100,4.88,light rain
4,Arraial Do Cabo,BR,-22.9661,-42.0278,63.23,56,99,21.70,overcast clouds
5,Busselton,AU,-33.6500,115.3333,49.08,70,85,26.44,overcast clouds
6,Tuatapere,NZ,-46.1333,167.6833,50.83,77,100,1.77,overcast clouds
7,Udankudi,IN,8.4333,78.0167,79.77,68,100,5.28,overcast clouds
8,Bredasdorp,ZA,-34.5322,20.0403,49.32,87,14,6.91,few clouds
9,Mataura,NZ,-46.1927,168.8643,50.49,78,100,3.47,overcast clouds


In [9]:
#create output file 
output_file = "Weather_Database/WeatherPy_Database.csv"

#export data to CSV file
city_data_df.to_csv(output_file, index_label="City_ID")